<a href="https://colab.research.google.com/github/skagnihotri/urban_sound_classification/blob/master/Urban_sound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#dataset downloaded
!wget -c https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz -O - | sudo tar -xz

--2019-08-12 19:42:40--  https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6023741708 (5.6G) [application/octet-stream]
Saving to: ‘STDOUT’

-                   100%[===================>]   5.61G  54.1MB/s    in 1m 43s  

2019-08-12 19:44:23 (55.9 MB/s) - written to stdout [6023741708/6023741708]



In [2]:
!apt-get install ffmpeg #backend for librosa

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [3]:
pip install librosa #sound preprocessing lib